In [ ]:
%%HTML
<style>
table.dataframe {
    border-collapse: collapse;
    border: 1px dashed #274472;
    font-size: 13px;
}

table.dataframe tr {
    border: 1px solid #274472;
}

table.dataframe td, table.dataframe th {
    margin: 0;    
    border: 1px solid #274472;
    background-color: #bfd7ed;
    padding-left: 0.25em;
    padding-right: 0.25em;
}

table.dataframe th:not(:empty) {
    border: 1px solid #274472;
    background-color: #bfd7ed;
    text-align: center;
    font-weight: normal;
}

table.dataframe tr:nth-child(2) th:empty {
    border-left: 1px solid #274472;
    border-right: 1px solid #274472;
}

table.dataframe td {
    border: 1px solid #274472;
    background-color: #bfd7ed;
}
</style>

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Standard plotly imports
import plotly.graph_objects as go
import plotly.express as px
import plotly.io as pio

pio.templates.default = "none"
colors = {
    'background': '#241b2f',
    'text': '#03edf9'
}

In [ ]:
fp = "bio_pop_co2_gdp_geo_final.csv"

df = pd.read_csv(fp)

df = df.rename(columns={'BIO':'Production'})

df['Income Set'] = np.where(df["Income Group"].str.contains("High", case=True, na=False), True, False)

df['Year End'] = pd.to_datetime(df['Year'], format="%Y")

df = df.drop("Unnamed: 11", axis=1)

df.head()

In [ ]:
xf = df.groupby(["Income Group"])["Production"].sum().sort_values().reset_index()
xf

In [ ]:
fig = px.bar(
    data_frame=xf,
    x="Income Group",
    y="Production",
)

fig.update_layout(plot_bgcolor="#ffffff", paper_bgcolor="#ffffff", autosize=False)


fig.show()

In [ ]:
fig = px.bar(
    data_frame=xf,
    x="Income Group",
    y="Production",
)

for template in ["plotly", "plotly_white", "plotly_dark", "ggplot2", "seaborn", "simple_white", "none"]:
    fig.update_layout(template=template, autosize=False, title="Biofuels: '%s' theme" % template)
    fig.show()

In [ ]:
xf = df.groupby(["Year","Income Group"])["Production"].sum().sort_index().reset_index()
xf['Production'] = xf['Production'].round(2)
xf.head()

In [ ]:
fig = px.bar(
    data_frame=xf, x="Year",
    y="Production",
    color="Income Group",
)

fig.update_layout(plot_bgcolor="#ffffff", paper_bgcolor="#ffffff", autosize=False)

fig.show()

In [ ]:
fig = px.bar(
    data_frame=xf,
    x="Year",
    y="Production",
    color="Income Group",
    barmode="group",
)

fig.update_layout(
    plot_bgcolor="#ffffff",
    paper_bgcolor="#ffffff",
    autosize=False,
)


fig.show()

In [ ]:
xf = df.groupby(["Year","Region"])["Production"].sum().reset_index()
xf['Production'] = xf['Production'].round(2)
xf.info()

In [ ]:
fig = px.scatter(
    data_frame=xf,
    x='Year',
    y='Production',
    color='Region',
)

fig.update_traces(mode='lines+markers')

fig.update_layout(autosize=False)

fig.show()

In [ ]:
#extract color palette, the palette can be changed
list_dist = list(set(xf['Region']))
pal = list(sns.color_palette(palette='Spectral',
                             n_colors=len(list_dist)).as_hex())
dict_color = dict(zip(list_dist, pal))

fig = px.bar(xf,
             x='Region',
             y='Production',
             color='Region',
             text='Production',
             color_discrete_map= dict_color,
             animation_frame='Year',
             animation_group='Region',
             range_y=[0, 1100],
             labels={ 'Production': 'Production Totals'},
            )
fig.update_layout(width=1000, height=600, showlegend=False,
                  xaxis = dict(tickmode = 'linear', dtick = 1))
fig.update_traces(textfont_size=12, textangle=0)
fig.show()

In [ ]:
q = df.groupby(['Year','Region'])['Production'].sum().unstack('Region')
q

In [ ]:
fig = px.scatter(
    data_frame=q,
    color='Region',
    hover_name='Region',
    facet_col='Region',
    facet_col_wrap=2,
    height=600,
    width=800,
)

fig.update_traces(mode='lines+markers')

In [ ]:
fig = px.scatter(
    data_frame=df.query("Year==2010"),
    x="CO2",
    y="GDP",
    size="Population",
    hover_name="Country Code",
    color="Income Group",
)

fig.update_layout(autosize=False)

fig.show()

In [ ]:
xf = df.groupby(['Year End', 'Country Code'])['Production'].sum().reset_index()

countries = ['USA', 'BRA', 'DEU', 'IND']

xf = xf[xf['Country Code'].isin(countries)].sort_values(['Country Code','Year End'])

xf.info()

In [ ]:
fig = px.area(
    data_frame=xf,
    x='Year End',
    y='Production',
    facet_col='Country Code',
    height=400,
    width=1200,
)
fig.update_layout(autosize=False)

fig.show()

In [ ]:
fig = px.histogram(
    data_frame=df,
    x="Production",
    width=650,
    height=400,
)

fig.show()

In [ ]:
fig = px.histogram(
    data_frame=df,
    x="Region",
    y="Production",
    width=800,
    height=400,
)

fig.show()

In [ ]:
fig = px.box(
    data_frame=df,
    x="Region",
    y="Production",
    width=800,
    height=400,
)

fig.show()

In [ ]:
fig = px.scatter_geo(
    data_frame=df,
    locations="Country Code",
    color="Income Group",
    hover_name="Country Name",
    size="Population",
    animation_frame="Year",
    projection="natural earth",
    width=800,
    height=400,
)

fig.show()

In [ ]:
ist = df.query("`Income Set`==True").groupby(["Year"])['Production'].sum().round(2).reset_index()
isf = df.query("`Income Set`==False").groupby(["Year"])['Production'].sum().round(2).reset_index()

display(ist)
display(isf)

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=ist.Year,
    y=ist.Production,
    name='High Income',
    marker_color='indianred')
)

fig.add_trace(go.Bar(
    x=isf.Year,
    y=isf.Production,
    name='Low Income',
    marker_color='lightsalmon')
)

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(
    autosize=False,
    barmode='group',
    xaxis_tickangle=-45,
    bargap=0.15,
    bargroupgap=0.1,
    plot_bgcolor='white',
)

fig.show()

In [ ]:
fig = go.Figure()

fig.add_trace(go.Bar(
    x=ist.Year,
    y=ist.Production,
    name='High Income',
    marker_color='indianred')
)

fig.add_trace(go.Bar(
    x=isf.Year,
    y=isf.Production,
    name='Low Income',
    marker_color='lightsalmon')
)

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(
    autosize=False,
    barmode='stack',
    xaxis_tickangle=-45,
    bargap=0.15,
    bargroupgap=0.1,
    plot_bgcolor='white',
)


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=ist.Year,
    y=ist.Production,
    name='High Income',
    marker_color='indianred')
)

fig.add_trace(go.Bar(
    x=isf.Year,
    y=isf.Production,
    name='Low Income',
    marker_color='lightsalmon')
)

# Here we modify the tickangle of the xaxis, resulting in rotated labels.
fig.update_layout(
    autosize=False,
    barmode='group',
    xaxis_tickangle=-45,
    bargap=0.15,
    bargroupgap=0.1,
    plot_bgcolor='white',
)


In [ ]:
colors = ['lightgray',] * isf.Year.size
colors[-1] = 'crimson'

fig = go.Figure(data=[go.Bar(
    x=isf.Year,
    y=isf.Production,
    marker_color=colors)]
)

from_yr = isf.Year.min()
to_yr = isf.Year.max()

fig.update_layout(
    title=f'<em>Low Income Production from {from_yr} to {to_yr}</em>',

    autosize=False,
    margin=dict(
        autoexpand=True,
        l=100,
        r=20,
        t=90,
    ),
    showlegend=False,
    plot_bgcolor='white'
)

fig.show()

## Sankey
---

In [ ]:
df = sns.load_dataset("penguins")
df = df.dropna(how='any')
df.head()

In [ ]:
df = df.groupby(['sex', 'species'])['body_mass_g'].mean()
df = df.to_frame(name='value').reset_index()
df.head()

In [ ]:
all_nodes = df.sex.values.tolist() + df.species.values.tolist()
source_indices = [all_nodes.index(sex) for sex in df.sex]
target_indices = [all_nodes.index(species) for species in df.species]
sankey_measure = df.value

fig = go.Figure(
    data=[go.Sankey(
        node=dict(
            pad=20,
            thickness=20,
            line=dict(color="black", width=1.0),
            label=all_nodes,
        ),
        link=dict(
            source=source_indices,
            target=target_indices,
            value=sankey_measure,
        )
    )]
)

fig.update_layout(title_text="Penguins", font_size=10, autosize=False)

fig.show()

## Plot Datasets
---

In [ ]:
d = pd.read_excel("plot_datasets.xlsx", sheet_name=None)

type(d)

In [ ]:
d.keys()

In [ ]:
for k, v in d.items():
    print(f"{k} is ", len(d[k]))

In [ ]:
df = d['str_line']
df

In [ ]:
df_props = {'background-color': 'white', 'color':'grey', 'font-family':'Lucida Sans', 'font-size':'13px', 'border': '1px dashed #000066'}

df.style.set_properties(**df_props)